In [1]:
import pickle
X_acl,X_ppg,y,y_participant,X_time = pickle.load(open('../data/tabular_data.p','rb'))

In [2]:
import numpy as np
X_ppg = X_ppg[:,np.arange(0,512,2),:]
X_all = np.concatenate([X_ppg,X_acl],axis=-1)

In [3]:
X_all.shape

(43363, 256, 8)

In [4]:
X_all = X_all[:,:,np.array([0,2,3,4])]
X_all.shape

(43363, 256, 4)

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,LeaveOneGroupOut,LeavePGroupsOut
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K 
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
%matplotlib notebook
from keras.layers import Conv1D,Attention,Reshape,BatchNormalization,TimeDistributed, \
Dropout,Input,MaxPooling1D,Flatten,Dense,Input, GaussianNoise,LSTM, Bidirectional, Input
from keras.models import Model, Sequential

In [6]:
logo = LeaveOneGroupOut()
logo = LeavePGroupsOut(n_groups=2)
for train_index, test_index in logo.split(X_all, y, y_participant.reshape(-1)):
    train_x, test_x_unseen = X_all[train_index], X_all[test_index]
    train_y, test_y_unseen = y[train_index], y[test_index]
    train_participant, test_participant_unseen = y_participant[train_index], y_participant[test_index]
    break
train_x.shape,train_y.shape,test_y_unseen.shape,test_x_unseen.shape

((37544, 256, 4), (37544,), (5819,), (5819, 256, 4))

In [7]:
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer,LabelEncoder

participant_ids = train_participant.copy()
train_participant = OneHotEncoder().fit_transform(train_participant.reshape(-1,1)).todense()

In [8]:
train_x, test_x, train_y, test_y,train_participant, test_participant,participant_ids_train, participant_ids_test = train_test_split(train_x,
                                                            train_y,
                                                            train_participant,
                                                            participant_ids,
                                                            test_size = 0.2,
                                                            random_state=42,
                                                            stratify=participant_ids)

In [9]:
train_x, val_x, train_y, val_y, train_participant,val_participant,participant_ids_train, participant_ids_val  = train_test_split(train_x,
                                                            train_y,
                                                            train_participant,
                                                            participant_ids_train,
                                                            test_size = 0.1,
                                                            random_state=42,
                                                            stratify=participant_ids_train)

In [10]:
train_x.shape,test_x.shape,val_x.shape,train_participant.shape,test_participant.shape,val_participant.shape, 

((27031, 256, 4),
 (7509, 256, 4),
 (3004, 256, 4),
 (27031, 13),
 (7509, 13),
 (3004, 13))

In [11]:
from keras_self_attention import SeqSelfAttention
from keras import backend as K
from keras.layers import Concatenate, Flatten,LeakyReLU, Activation, RepeatVector, Permute, Multiply, Lambda, Dense, merge# Define a regular layer instead of writing a custom layer

In [12]:
def get_model(input_shape=(256,1),act='relu',loss="categorical_crossentropy",opt='adam',n_classes=350):
    model =  Sequential()
    model.add(Conv1D(100,10,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Conv1D(50,2,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Flatten())
    model.add(Dense(n_classes,activation='sigmoid',kernel_initializer='normal'))
    model.add(Dense(n_classes,activation='softmax',kernel_initializer='normal'))
    model.compile(loss=loss,optimizer=opt,metrics=['acc'])
    return model

In [13]:
n_classes = len(np.unique(participant_ids_train))
model = get_model(input_shape=(256,4),n_classes=n_classes)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 256, 100)          4100      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 128, 100)          0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 100)          400       
_________________________________________________________________
dropout (Dropout)            (None, 128, 100)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 128, 100)          100100    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 64, 100)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 100)           4

In [14]:
from keras.models import load_model
filepath = '../model_files/base_cnn_8_seconds_ppg_acl_wesad.hdf5'
# model = load_model(filepath)

In [15]:
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40)
callbacks_list = [es,checkpoint]
history = model.fit(train_x,train_participant,validation_data=(val_x,val_participant), epochs=400, batch_size=200,
          callbacks=callbacks_list,shuffle=True)

Epoch 1/400
136/136 [==============================] - ETA: 0s - loss: 2.3431 - acc: 0.2637
Epoch 00001: val_loss improved from inf to 2.17882, saving model to ../model_files/base_cnn_8_seconds_ppg_acl_wesad.hdf5
136/136 [==============================] - 2s 15ms/step - loss: 2.3431 - acc: 0.2637 - val_loss: 2.1788 - val_acc: 0.3123
Epoch 2/400
136/136 [==============================] - ETA: 0s - loss: 2.0039 - acc: 0.3925
Epoch 00002: val_loss improved from 2.17882 to 1.89207, saving model to ../model_files/base_cnn_8_seconds_ppg_acl_wesad.hdf5
136/136 [==============================] - 2s 13ms/step - loss: 2.0039 - acc: 0.3925 - val_loss: 1.8921 - val_acc: 0.4008
Epoch 3/400
135/136 [============================>.] - ETA: 0s - loss: 1.7991 - acc: 0.4437
Epoch 00003: val_loss improved from 1.89207 to 1.72436, saving model to ../model_files/base_cnn_8_seconds_ppg_acl_wesad.hdf5
136/136 [==============================] - 2s 13ms/step - loss: 1.7990 - acc: 0.4437 - val_loss: 1.7244 - val

Epoch 26/400
136/136 [==============================] - ETA: 0s - loss: 0.2649 - acc: 0.9238
Epoch 00026: val_loss improved from 0.31721 to 0.28633, saving model to ../model_files/base_cnn_8_seconds_ppg_acl_wesad.hdf5
136/136 [==============================] - 2s 13ms/step - loss: 0.2649 - acc: 0.9238 - val_loss: 0.2863 - val_acc: 0.9154
Epoch 27/400
136/136 [==============================] - ETA: 0s - loss: 0.2581 - acc: 0.9241
Epoch 00027: val_loss did not improve from 0.28633
136/136 [==============================] - 2s 12ms/step - loss: 0.2581 - acc: 0.9241 - val_loss: 0.3201 - val_acc: 0.9068
Epoch 28/400
136/136 [==============================] - ETA: 0s - loss: 0.2364 - acc: 0.9324
Epoch 00028: val_loss improved from 0.28633 to 0.25973, saving model to ../model_files/base_cnn_8_seconds_ppg_acl_wesad.hdf5
136/136 [==============================] - 2s 13ms/step - loss: 0.2364 - acc: 0.9324 - val_loss: 0.2597 - val_acc: 0.9238
Epoch 29/400
136/136 [==============================] 

136/136 [==============================] - ETA: 0s - loss: 0.1077 - acc: 0.9660
Epoch 00054: val_loss improved from 0.16319 to 0.16001, saving model to ../model_files/base_cnn_8_seconds_ppg_acl_wesad.hdf5
136/136 [==============================] - 2s 13ms/step - loss: 0.1077 - acc: 0.9660 - val_loss: 0.1600 - val_acc: 0.9511
Epoch 55/400
136/136 [==============================] - ETA: 0s - loss: 0.1080 - acc: 0.9674
Epoch 00055: val_loss improved from 0.16001 to 0.15247, saving model to ../model_files/base_cnn_8_seconds_ppg_acl_wesad.hdf5
136/136 [==============================] - 2s 13ms/step - loss: 0.1080 - acc: 0.9674 - val_loss: 0.1525 - val_acc: 0.9537
Epoch 56/400
136/136 [==============================] - ETA: 0s - loss: 0.1400 - acc: 0.9559
Epoch 00056: val_loss did not improve from 0.15247
136/136 [==============================] - 2s 12ms/step - loss: 0.1400 - acc: 0.9559 - val_loss: 0.2151 - val_acc: 0.9341
Epoch 57/400
136/136 [==============================] - ETA: 0s - l

136/136 [==============================] - ETA: 0s - loss: 0.0708 - acc: 0.9779
Epoch 00083: val_loss did not improve from 0.11681
136/136 [==============================] - 2s 12ms/step - loss: 0.0708 - acc: 0.9779 - val_loss: 0.1269 - val_acc: 0.9604
Epoch 84/400
136/136 [==============================] - ETA: 0s - loss: 0.0601 - acc: 0.9815
Epoch 00084: val_loss improved from 0.11681 to 0.10893, saving model to ../model_files/base_cnn_8_seconds_ppg_acl_wesad.hdf5
136/136 [==============================] - 2s 13ms/step - loss: 0.0601 - acc: 0.9815 - val_loss: 0.1089 - val_acc: 0.9654
Epoch 85/400
136/136 [==============================] - ETA: 0s - loss: 0.0709 - acc: 0.9776
Epoch 00085: val_loss did not improve from 0.10893
136/136 [==============================] - 2s 12ms/step - loss: 0.0709 - acc: 0.9776 - val_loss: 0.1143 - val_acc: 0.9660
Epoch 86/400
136/136 [==============================] - ETA: 0s - loss: 0.0682 - acc: 0.9785
Epoch 00086: val_loss did not improve from 0.108

136/136 [==============================] - 2s 13ms/step - loss: 0.0577 - acc: 0.9815 - val_loss: 0.0998 - val_acc: 0.9647
Epoch 113/400
136/136 [==============================] - ETA: 0s - loss: 0.0521 - acc: 0.9841
Epoch 00113: val_loss did not improve from 0.09981
136/136 [==============================] - 2s 12ms/step - loss: 0.0521 - acc: 0.9841 - val_loss: 0.1031 - val_acc: 0.9667
Epoch 114/400
136/136 [==============================] - ETA: 0s - loss: 0.0524 - acc: 0.9841
Epoch 00114: val_loss did not improve from 0.09981
136/136 [==============================] - 2s 12ms/step - loss: 0.0524 - acc: 0.9841 - val_loss: 0.1035 - val_acc: 0.9704
Epoch 115/400
136/136 [==============================] - ETA: 0s - loss: 0.0481 - acc: 0.9850
Epoch 00115: val_loss improved from 0.09981 to 0.09917, saving model to ../model_files/base_cnn_8_seconds_ppg_acl_wesad.hdf5
136/136 [==============================] - 2s 13ms/step - loss: 0.0481 - acc: 0.9850 - val_loss: 0.0992 - val_acc: 0.9700
Epo

Epoch 142/400
136/136 [==============================] - ETA: 0s - loss: 0.0380 - acc: 0.9882
Epoch 00142: val_loss did not improve from 0.07897
136/136 [==============================] - 2s 12ms/step - loss: 0.0380 - acc: 0.9882 - val_loss: 0.0807 - val_acc: 0.9750
Epoch 143/400
136/136 [==============================] - ETA: 0s - loss: 0.0475 - acc: 0.9849
Epoch 00143: val_loss did not improve from 0.07897
136/136 [==============================] - 2s 12ms/step - loss: 0.0475 - acc: 0.9849 - val_loss: 0.0845 - val_acc: 0.9770
Epoch 144/400
136/136 [==============================] - ETA: 0s - loss: 0.0463 - acc: 0.9863
Epoch 00144: val_loss did not improve from 0.07897
136/136 [==============================] - 2s 12ms/step - loss: 0.0463 - acc: 0.9863 - val_loss: 0.0912 - val_acc: 0.9737
Epoch 145/400
136/136 [==============================] - ETA: 0s - loss: 0.0446 - acc: 0.9854
Epoch 00145: val_loss did not improve from 0.07897
136/136 [==============================] - 2s 12ms/ste

Epoch 173/400
136/136 [==============================] - ETA: 0s - loss: 0.0387 - acc: 0.9881
Epoch 00173: val_loss did not improve from 0.06342
136/136 [==============================] - 2s 12ms/step - loss: 0.0387 - acc: 0.9881 - val_loss: 0.0973 - val_acc: 0.9737
Epoch 174/400
136/136 [==============================] - ETA: 0s - loss: 0.0405 - acc: 0.9863
Epoch 00174: val_loss did not improve from 0.06342
136/136 [==============================] - 2s 12ms/step - loss: 0.0405 - acc: 0.9863 - val_loss: 0.0831 - val_acc: 0.9780
Epoch 175/400
136/136 [==============================] - ETA: 0s - loss: 0.0379 - acc: 0.9882
Epoch 00175: val_loss did not improve from 0.06342
136/136 [==============================] - 2s 12ms/step - loss: 0.0379 - acc: 0.9882 - val_loss: 0.0836 - val_acc: 0.9780
Epoch 176/400
136/136 [==============================] - ETA: 0s - loss: 0.0360 - acc: 0.9880
Epoch 00176: val_loss did not improve from 0.06342
136/136 [==============================] - 2s 12ms/ste

136/136 [==============================] - ETA: 0s - loss: 0.0299 - acc: 0.9897
Epoch 00203: val_loss improved from 0.05850 to 0.05846, saving model to ../model_files/base_cnn_8_seconds_ppg_acl_wesad.hdf5
136/136 [==============================] - 2s 13ms/step - loss: 0.0299 - acc: 0.9897 - val_loss: 0.0585 - val_acc: 0.9857
Epoch 204/400
136/136 [==============================] - ETA: 0s - loss: 0.0276 - acc: 0.9919
Epoch 00204: val_loss did not improve from 0.05846
136/136 [==============================] - 2s 12ms/step - loss: 0.0276 - acc: 0.9919 - val_loss: 0.0727 - val_acc: 0.9774
Epoch 205/400
136/136 [==============================] - ETA: 0s - loss: 0.0293 - acc: 0.9907
Epoch 00205: val_loss did not improve from 0.05846
136/136 [==============================] - 2s 12ms/step - loss: 0.0293 - acc: 0.9907 - val_loss: 0.0587 - val_acc: 0.9827
Epoch 206/400
136/136 [==============================] - ETA: 0s - loss: 0.0504 - acc: 0.9844
Epoch 00206: val_loss did not improve from 0.

Epoch 233/400
136/136 [==============================] - ETA: 0s - loss: 0.0259 - acc: 0.9921
Epoch 00233: val_loss did not improve from 0.04871
136/136 [==============================] - 2s 12ms/step - loss: 0.0259 - acc: 0.9921 - val_loss: 0.0487 - val_acc: 0.9837
Epoch 234/400
136/136 [==============================] - ETA: 0s - loss: 0.0251 - acc: 0.9922
Epoch 00234: val_loss did not improve from 0.04871
136/136 [==============================] - 2s 12ms/step - loss: 0.0251 - acc: 0.9922 - val_loss: 0.0559 - val_acc: 0.9840
Epoch 235/400
136/136 [==============================] - ETA: 0s - loss: 0.0227 - acc: 0.9929
Epoch 00235: val_loss improved from 0.04871 to 0.04806, saving model to ../model_files/base_cnn_8_seconds_ppg_acl_wesad.hdf5
136/136 [==============================] - 2s 13ms/step - loss: 0.0227 - acc: 0.9929 - val_loss: 0.0481 - val_acc: 0.9854
Epoch 236/400
136/136 [==============================] - ETA: 0s - loss: 0.0237 - acc: 0.9924
Epoch 00236: val_loss did not i

136/136 [==============================] - ETA: 0s - loss: 0.0277 - acc: 0.9907
Epoch 00263: val_loss did not improve from 0.04635
136/136 [==============================] - 2s 12ms/step - loss: 0.0277 - acc: 0.9907 - val_loss: 0.0576 - val_acc: 0.9827
Epoch 264/400
136/136 [==============================] - ETA: 0s - loss: 0.0223 - acc: 0.9929
Epoch 00264: val_loss did not improve from 0.04635
136/136 [==============================] - 2s 12ms/step - loss: 0.0223 - acc: 0.9929 - val_loss: 0.0549 - val_acc: 0.9844
Epoch 265/400
136/136 [==============================] - ETA: 0s - loss: 0.0260 - acc: 0.9917
Epoch 00265: val_loss did not improve from 0.04635
136/136 [==============================] - 2s 12ms/step - loss: 0.0260 - acc: 0.9917 - val_loss: 0.0500 - val_acc: 0.9840
Epoch 266/400
136/136 [==============================] - ETA: 0s - loss: 0.0271 - acc: 0.9916
Epoch 00266: val_loss did not improve from 0.04635
136/136 [==============================] - 2s 12ms/step - loss: 0.02

In [16]:
model = load_model(filepath)
test_participant_pred = model.predict(test_x)

In [17]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#filtered_acl

In [ ]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#filtered ppg+filtered_acl

In [ ]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#filtered ppg+acl

In [ ]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#filtered ppg

In [18]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#ppg+acl

[[517   0   2   0   0   0   0   3   0   2   0   0   0]
 [  0 647   0   0   1   0   0   0   0   0   0   0   0]
 [  4   0 508   1   0   0   0   3   6   0   0   1   0]
 [  1   0   0 701   0   0   0   0   1   3   0   0   0]
 [  0  13   0   0 576   0   0   0   0   0   0   0   2]
 [  0   0   0   0   0 548   1   0   3   1   0   0   0]
 [  0   0   0   0   0   0 540   1   8   0   0   0   0]
 [  1   0   1   0   0   0   3 613   0   4   0   3   0]
 [  2   0   1   1   0   0   0   0 556   0   0   0   2]
 [  4   0   2   2   0   0   3   2   1 502   2   3   0]
 [  0   0   0   1   0   3   1   2   1   3 531   3   1]
 [  1   0   0   2   0   2   0   4   3   1   1 593   0]
 [  0   2   0   0   0   0   0   0   0   0   0   0 552]]
0.9833533093620989


In [ ]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#ppg

In [ ]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#acl